# Visualize RCD Effect

---

**Disclaimer**: This script is for demonstration purposes only. It is not intended to assess or verify its functionality on any other devices or systems.

samples used in paper:
chair -> ./1015e71a0d21b127de03ab2a27ba7531.pcd (); ./1016f4debe988507589aae130c1f06fb.pcd(4, 0, 1)


In [ ]:
import os
import numpy as np
import torch
import cv2 as cv

import open3d as o3d
# import visulization libs
import visdom
vis = visdom.Visdom()

import sys
sys.path.append("../")
from extensions.chamfer_dist import ChamferDistanceL2, PatialChamferDistanceL2, RegionAwareChamferDistance

import numexpr as ne
os.environ['NUMEXPR_MAX_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

np.random.seed(0)

In [ ]:
# complete = o3d.io.read_point_cloud("./samples/rcd/chair2/1016f4debe988507589aae130c1f06fb.pcd")
# o3d.io.write_point_cloud("./samples/rcd/chair2/complete.ply", complete)
# partial = o3d.io.read_point_cloud("./samples/rcd/chair2/04.pcd")
# o3d.io.write_point_cloud("./samples/rcd/chair2/04.ply", partial)

complete = o3d.io.read_point_cloud("./samples/rcd/chair2/complete.ply")
partial = o3d.io.read_point_cloud("./samples/rcd/chair2/04.ply")
# o3d.visualization.draw_geometries([complete, partial])

inpt = np.asarray(partial.points)
gt = np.asarray(complete.points)
choice = np.random.choice(gt.shape[0], 8192, replace=False)
gt = gt[choice]
gt.shape, inpt.shape

In [ ]:
vis.scatter(X=inpt, win='partial', opts=dict(title='partial', markersize=2))
vis.scatter(X=gt, win='complete', opts=dict(title='complete', markersize=2))

In [ ]:
print(gt.shape, inpt.shape)
# randomly select 3 x 2048 points to add different scales of noise
idx = np.random.choice(gt.shape[0], 3*2048, replace=False)
# sigma1 = 0.0001
sigma2 = 0.001
sigma3 = 0.018
# noise = torch.normal(torch.zeros((2048, 3)), sigma1 * torch.ones((2048, 3)))
synthetic_pred = torch.from_numpy(gt).float()
# synthetic_pred[idx[:2048]] += noise
noise = torch.normal(torch.zeros((2048, 3)), sigma2 * torch.ones((2048, 3)))
synthetic_pred[idx[2048:4096]] += noise
noise = torch.normal(torch.zeros((2048, 3)), sigma3 * torch.ones((2048, 3)))
synthetic_pred[idx[4096:]] += noise

vis.scatter(X=synthetic_pred.numpy(), win='synthetic_pred', opts=dict(title='synthetic_pred', markersize=2))

In [ ]:
pred, target = synthetic_pred.unsqueeze(0), torch.from_numpy(inpt).unsqueeze(0)
pred.shape, target.shape

In [ ]:
# implement the distance function

def pairwise_distance(x, y):
    """
    compute the pairwise distance between x and y and return the shortest distance for each element in x
    Args:
        x: (B, N, C)
        y: (B, M, C)
    Returns:
        dist: (B, N, M)
    """
    x = x.unsqueeze(2)
    y = y.unsqueeze(1)
    dist = torch.norm((x - y), dim=-1)
    return dist

In [ ]:
# Chamfer Distance

def per_point_chamfer_distance(x, y):
    """
    compute the chamfer distance between x and y
    Args:
        x: (B, N, C); complete prediction
        y: (B, M, C); partial input
    Returns:
        dist: (B, N, M)
    """
    # for each point in x, find the closest point in y
    dist1 = pairwise_distance(x, y)
    dist1 = torch.min(dist1, dim=-1)[0]
    dist1 = dist1.flatten()

    # for each point in y, find the closest point idx in x
    dist2 = pairwise_distance(y, x)
    dist2, idx = torch.min(dist2, dim=-1)

    for i in idx.unique():
        reverse_dist = dist2[idx.eq(i)].mean()
        dist1[i] += reverse_dist
        dist1[i] /= 2

    return dist1

In [ ]:
cd = per_point_chamfer_distance(pred, target)

In [ ]:
density = cd.numpy()
density = (density - density.min())/ (density.max() - density.min())
density = 255 - np.uint8(255*density)
color_map = cv.applyColorMap(density, cv.COLORMAP_JET)/255
color_map = np.squeeze(color_map)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(synthetic_pred.numpy())
pcd.colors = o3d.utility.Vector3dVector(color_map)
o3d.visualization.draw_geometries([pcd])

In [ ]:
def per_point_unidirectional_chamfer_distance(x, y):
    """
    compute the per point unidirectional chamfer distance between x and y
    Args:
        x: (B, N, C); complete prediction
        y: (B, M, C); partial input
    Returns:
        dist: (B, N, M)
    """
    # for each point in y, find the closest point idx in x
    dist1 = np.zeros(x.shape[1])
    dist2 = pairwise_distance(y, x)
    dist2, idx = torch.min(dist2, dim=-1)

    for i in idx.unique():
        reverse_dist = dist2[idx.eq(i)].mean()
        dist1[i] += reverse_dist

    return torch.Tensor(dist1)

In [ ]:
ucd = per_point_unidirectional_chamfer_distance(pred, target)
density = ucd.numpy()
density = (density - density.min())/ (density.max() - density.min())
density = 255 - np.uint8(255*density)
color_map = cv.applyColorMap(density, cv.COLORMAP_JET)/255
color_map = np.squeeze(color_map)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(synthetic_pred.numpy())
pcd.colors = o3d.utility.Vector3dVector(color_map)
o3d.visualization.draw_geometries([pcd])


In [ ]:
def per_point_region_aware_chamfer_distance(x, y, n_nbrs=64):
    y2x_dist = - pairwise_distance(y, x)
    y2x_dist, y2x_idx = torch.topk(y2x_dist, k=n_nbrs, dim=-1)
    y2x_dist = - y2x_dist

    effect = np.zeros(x.shape[1])
    selected_points = y2x_idx.unique()
    for i in selected_points:
        effect[i] = torch.norm(x[:, i, :]-y, dim=-1).min()
        effect[i] += y2x_dist[y2x_idx.eq(i)].mean()
        effect[i] /= 2
    return torch.Tensor(effect)

In [ ]:
rcd = per_point_region_aware_chamfer_distance(pred, target, n_nbrs=128)

In [ ]:
density = rcd.numpy()
density = (density - density.min())/ (density.max() - density.min())
density = 255 - np.uint8(255*density)
color_map = cv.applyColorMap(density, cv.COLORMAP_JET)/255
color_map = np.squeeze(color_map)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(synthetic_pred.numpy())
pcd.colors = o3d.utility.Vector3dVector(color_map)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# assign RGB 211,211,211 to partial
pcd = o3d.geometry.PointCloud()
points = np.asarray(partial.points)
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(np.ones((points.shape[0], 3))*128/255)
o3d.visualization.draw_geometries([pcd])

In [ ]:
# camera pose, copy all and paste to meshlab
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" :
	[
		{
			"boundingbox_max" : [ 0.27304661273956299, 0.4258209764957428, 0.25082644820213318 ],
			"boundingbox_min" : [ -0.28748518228530884, -0.41839838027954102, -0.24128541350364685 ],
			"field_of_view" : 60.0,
			"front" : [ 0.58236521851613265, 0.2631111520757079, -0.76917051030057748 ],
			"lookat" : [ -0.0072192847728729248, 0.0037112981081008911, 0.0047705173492431641 ],
			"up" : [ -0.18422289729433594, 0.96427264666648294, 0.19036855571048716 ],
			"zoom" : 0.88000000000000012
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}